In [4]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [42]:
df = (
    pl.scan_csv("../data/processed/df_procesada.csv", separator=";")
    .filter((pl.col("COD_HOSPITAL") == 112103) & (pl.col("TIPO_ACTIVIDAD") == "HOSPITALIZACIÓN"))
    .with_columns(pl.col("DIAGNOSTICO1").str.replace(".", "", literal=True).str.pad_end(4, "X"))
    .group_by(["ANIO_EGRESO", "DIAGNOSTICO1"])
    .agg(
        [
            pl.col("CIP_ENCRIPTADO").n_unique().alias("n_pacientes_distintos"),
            pl.len().alias("n_egresos"),
            pl.col("IR_29301_PESO").mean().round(4).alias("peso_grd_promedio"),
            pl.col("ESTANCIA").sum().alias("dias_estada_totales"),
        ]
    )
    .sort(by=["ANIO_EGRESO", "n_egresos"], descending=True)
)

In [43]:
df = df.collect(streaming=True)

In [36]:
df.write_excel("../data/interim/resumen_egresos_INT_grd.xlsx")